#### Importing library

In [46]:
import os
import numpy as np

In [47]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

#### set up path and labelling

In [48]:
data_path = os.path.join('30_data_fall_reverse')
actions = np.array(['falling0', 'lying', 'sitting', 'standing'])

no_videos = 400
video_length = 30

In [49]:
label_map = {label:num for num, label in enumerate(actions)}

In [50]:
label_map

{'falling0': 0, 'lying': 1, 'sitting': 2, 'standing': 3}

#### labeling data


In [51]:
sequences, labels = [], []

##looping through actions folder
for action in actions:

    ##Goes into each action folder and 
    ##loops through each video folder by name (assumes folder names are numbers like 0, 1, 2, ...)
    
    path_for_load = os.path.join(data_path, action)
    content = np.array(os.listdir(path_for_load)).astype(int)

    for sequence in content:

        ##load each npy file into window[]
        window = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(data_path, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)

        sequences.append(window)
        labels.append(label_map[action])

In [53]:
np.array(sequences).shape

(1600, 30, 132)

#### test up for training

In [54]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

#### splitting data, 60% training, 20% validation, 20% testing

In [55]:
# First split: 60% train, 40% temp
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Second split: split the 40% temp into 20% val and 20% test (i.e., 50% of temp)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#### Library for building LSTM model

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, Callback


#### set up tensorboard for live monitoring of training

In [66]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [67]:
X.shape

(1600, 30, 132)

#### Build model

In [68]:
model = Sequential()
model.add(Input(shape=(30, 132)))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

#### Compile model and training logic

In [ ]:
## basic model complie
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## I didn't code this out, I got some help from GPT.
## why, when training, I notice at a certian epoch, the accuracy got very consistant
## then it plunges to down from 0.94 to 0.7 or 0.5

## this script stop the training when the accuaracy is stabilizes
class AccuracyStabilizer(Callback):
    def __init__(self, min_delta=0.001, patience=25):
        super().__init__()             # Call parent class constructor
        self.min_delta = min_delta     # Minimum required change in accuracy to count as "progress"
        self.patience = patience       # Number of epochs to check for stability
        self.acc_history = []          # Store training accuracy over time

    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get("accuracy")     # Get the training accuracy at the end of the epoch
        if acc is None:
            return                     # If accuracy is not available, skip this epoch

        self.acc_history.append(acc)   # Save the accuracy value

        # Only check if we have enough history (at least `patience` entries)
        if len(self.acc_history) > self.patience:
            recent = self.acc_history[-self.patience:]   # Get the most recent N accuracies
            deltas = np.abs(np.diff(recent))             # Compute the changes between them

            # If all changes are smaller than the threshold (i.e. stable), stop training

            if np.all(deltas < self.min_delta):
                print(f"\nStopping training: accuracy change < {self.min_delta} for {self.patience} epochs.")
                self.model.stop_training = True           # Tell Keras to stop training

In [ ]:
stabilizer = AccuracyStabilizer(min_delta=0.001, patience=10)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    callbacks=[tb_callback, stabilizer],
    batch_size=32
)

Epoch 1/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.3683 - loss: 1.3538 - val_accuracy: 0.5000 - val_loss: 1.1434
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5484 - loss: 1.0727 - val_accuracy: 0.6219 - val_loss: 0.8737
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5508 - loss: 1.0192 - val_accuracy: 0.5437 - val_loss: 0.9916
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6161 - loss: 0.9149 - val_accuracy: 0.6594 - val_loss: 0.8032
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6925 - loss: 0.7726 - val_accuracy: 0.7375 - val_loss: 0.6430
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7826 - loss: 0.5862 - val_accuracy: 0.7469 - val_loss: 0.6290
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7553 - loss: 0.5748 - val_accuracy: 0.7219 - val_loss: 0.7185
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.7786 - loss: 0.5765 - val_accuracy: 0.

In [72]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 30, 64)         │        50,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,028 (800.89 KB)

 Trainable params: 205,028 (800.89 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8227 - loss: 0.9481
Test Accuracy: 0.8375


In [76]:
model.save('new_model2.keras')